In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import torch
import math

In [2]:
from __future__ import print_function
import torch
import numpy as np
import torch.nn.functional as F

import warnings
warnings.filterwarnings('ignore')

dtype = torch.float
device = torch.device("cpu")

In [3]:
import sys
sys.path.append('../')
from utils import data_processing as proc
path="./data/last3years.csv"


In [4]:
#prepare_tabular_data_vision example 
e,d = proc.prepare_tabular_data_vision(path="../data/last3years.csv", min_wind=34, min_steps=20,
                  max_steps=120, get_displacement=True, forecast=False, predict_period = 24, one_hot = True)

after fillna, nan entries of _WIND is 648
after fillna, nan entries of _PRES is 537
after interpolate, nan entries of LAT is 0
after interpolate, nan entries of LON is 0
after interpolate, nan entries of WMO_WIND is 283
after interpolate, nan entries of WMO_PRES is 239
after interpolate, nan entries of DIST2LAND is 0
after interpolate, nan entries of STORM_SPEED is 0
selecting storms according to min_wind, min_step, max_step requirements
nan entries of LAT is 0
nan entries of LON is 0
nan entries of WMO_WIND is 241
nan entries of WMO_PRES is 129
nan entries of DIST2LAND is 0
nan entries of STORM_SPEED is 0
one-hot added for  BASIN
The dictionary of storms has been created.
The trajectories have now been padded.
There are 375 storms with 26 features, and maximum number of steps is 120 and minimum is 120.
creating tensor, dropping SID and ISO_TIME features
The tensor has now been created.
columns in tensor: ['SID', 'NAME', 'ISO_TIME', 'LAT', 'LON', 'WMO_WIND', 'WMO_PRES', 'DIST2LAND', 'S

In [14]:
#here set up the path of your file, a priori it should be on same directory
data = pd.read_csv("../data/last3years.csv")
data.drop(0, axis=0, inplace=True)
# Preview the first 5 lines of the loaded data 
# data.head()

In [15]:
#convert columns to numeric values
#and interpolate missing values
def numeric_data(data):
    for i in ['LAT', 'LON', 'WMO_WIND', 'WMO_PRES', 'DIST2LAND', 'STORM_SPEED','STORM_DIR']:
        data[i]=pd.to_numeric(data[i],errors='coerce').astype('float64')
    return data

#fillna of _WIND and _PRES:
def fillna_wind_pres(df):
    #substitute source of information
    sub_list = ['USA','TOKYO','CMA','HKO','NEWDELHI','MLC','TD9635','NEUMANN','DS824','TD9636','WELLINGTON','NADI']
    #for two features 
    for feature in ['_WIND','_PRES']:
        #loop over sub sources in order 
        for sub in sub_list:
            col_sub = pd.to_numeric(df[sub+feature],errors='coerce').astype('float64')
            df['WMO'+feature].fillna(col_sub, inplace=True)
        print('after fillna, nan entries of %s is %s'%(feature, df.loc[df['WMO'+feature].isnull()].shape[0]))
    return df 

def interpolate_data(data):
    for i in ['LAT', 'LON', 'WMO_WIND', 'WMO_PRES', 'DIST2LAND', 'STORM_SPEED']:
        #use the linear interpolation value to fillna, fill 2 value on both direction 
        data[i]=data[i].interpolate(method='linear', limit=2, limit_direction='both')
        print('after interpolate, nan entries of %s is %s'%(i, data.loc[data[i].isnull()].shape[0]))
    return data 


# df0 is  cleaned data 
data = numeric_data(data)
data = fillna_wind_pres(data)
# data = interpolate_data(data)
# data.head(10)

after fillna, nan entries of _WIND is 648
after fillna, nan entries of _PRES is 537


In [48]:
def interpolate(data):
    for i in ['LAT', 'LON', 'WMO_WIND', 'WMO_PRES', 'DIST2LAND', 'STORM_SPEED']:
        #use the linear interpolation value to fillna, fill 2 value on both direction 
        data[i]=data[i].interpolate(method='linear', limit=2, limit_direction='forward')
    return data 

def check_nan(data):
    for i in ['LAT', 'LON', 'WMO_WIND', 'WMO_PRES', 'DIST2LAND', 'STORM_SPEED']:
            print('nan entries of %s is %s'%(i, data.loc[data[i].isnull()].shape[0]))
    return 

# function to select storms based on specified requirements   
def select_storms(data, min_wind=34, min_steps = 20, max_steps = 120):
    print('selecting storms according to min_wind, min_step, max_step requirements')
    #get unique storm_id:
    SID=pd.unique(data['SID']).tolist()
    #create empty dictionary
    dict0={}
    ind = 0
    for i in range(len(SID)):
        #get data of a particular SID
        M = data.loc[data['SID'] == SID[i]]
        M.reset_index(inplace=True, drop=True)
        #use the linear interpolation value to fillna 
        M= interpolate(M) 
        length = M.loc[M['WMO_WIND'] >= min_wind].shape[0] 
        #if there is any index than min_wind 
        if length > min_steps: 
            #first index 
            i0 = M.index[M['WMO_WIND']>= min_wind][0] 
            i_max = min(max_steps, length)
            M_selected = M.iloc[i0:i_max]
            #record selected data 
            dict0.update({ind: M_selected})
            ind +=1 
            
    print('number of storms selected is', ind) 
    
    #concatenate dict to df 
    data_out = pd.DataFrame()
    for i in dict0:
        data_out = pd.concat([data_out, dict0[i]], axis=0)
    #reset index 
    data_out.reset_index(inplace=True, drop=True)
    #check nan entries 
    check_nan(data_out)
    #fillna with zero 
    data_out.fillna(value=0, inplace=True)
    return data_out 

data_selected= select_storms(data)

selecting storms according to min_wind, min_step, max_step requirements
number of storms selected is 240
nan entries of LAT is 0
nan entries of LON is 0
nan entries of WMO_WIND is 20
nan entries of WMO_PRES is 20
nan entries of DIST2LAND is 0
nan entries of STORM_SPEED is 0


In [47]:
# data_selected.loc[data_selected['WMO_WIND'].isnull()]
data_selected['WMO_WIND']

0        37.0
1        45.0
2        45.0
3        45.0
4        45.0
        ...  
9830     85.0
9831     97.0
9832    110.0
9833    112.0
9834    115.0
Name: WMO_WIND, Length: 9835, dtype: float64

In [9]:
def check_nan(data):
    for i in ['LAT', 'LON', 'WMO_WIND', 'WMO_PRES', 'DIST2LAND', 'STORM_SPEED']:
            print('nan entries of %s is %s'%(i, data.loc[data[i].isnull()].shape[0]))
    return 
check_nan(data)

nan entries of LAT is 0
nan entries of LON is 0
nan entries of WMO_WIND is 486
nan entries of WMO_PRES is 3048
nan entries of DIST2LAND is 0
nan entries of STORM_SPEED is 0


In [40]:
df=df0.iloc[1:].copy()
# pd.to_datetime(df['ISO_TIME'], format= '%Y-%m-%d %H:%M:%S')

def smooth_day(df):
    df['ISO_TIME'] = pd.to_datetime(df['ISO_TIME'], format= '%Y-%m-%d %H:%M:%S')
    df['cos_day'] = np.cos(2 * np.pi * df['ISO_TIME'].dt.day / 365)
    df['sign_day'] = 0 
    df.loc[(df['ISO_TIME'].dt.hour <=11) & (df['ISO_TIME'].dt.hour >=0),'sign_day'] = 1
    return df

smooth_day(df).head()
# df.loc[(df['ISO_TIME'].dt.hour <=11) & (df['ISO_TIME'].dt.hour >=0)]['SID']=1

,SID,NUMBER,ISO_TIME,LAT,LON,WMO_WIND,WMO_PRES,DIST2LAND,STORM_SPEED,cos_day,sign_day
1,2016005N02187,1,2016-01-05 06:00:00,2.0000,-173.500,25.0,1006.0,1738.0,3.0,0.996298,1
2,2016005N02187,1,2016-01-05 09:00:00,2.0450,-173.353,25.0,1006.0,1737.0,3.0,0.996298,1
3,2016005N02187,1,2016-01-05 12:00:00,2.1000,-173.200,25.0,1006.0,1748.0,3.0,0.996298,0
4,2016005N02187,1,2016-01-05 15:00:00,2.1775,-173.042,25.0,1005.5,1759.0,4.0,0.996298,0
5,2016005N02187,1,2016-01-05 18:00:00,2.3000,-172.900,25.0,1005.0,1770.0,4.0,0.996298,0


In [5]:
#to have one-hot encoding of basin and nature of the storm
def add_one_hot(data, df0):
    basin = pd.get_dummies(data['BASIN'],prefix='basin')
    basin.drop(columns=['basin_ '], inplace = True)
    nature = pd.get_dummies(data['NATURE'],prefix='nature')
    nature.drop('nature_ ', axis=1, inplace = True)
    frames = [df0, basin, nature]
    df0 = pd.concat(frames, axis = 1)
    print("Basin and Nature of the storm are now added and one-hot.")
    return df0

In [6]:
#This code allows to get the maximum wind change in the last X hours.
def get_max_change(data, time, i):
    t = time//3
    try:
        val = max(data['WMO_WIND'][i-t:i])-min(data['WMO_WIND'][i-t:i])
    except:
        val = 'NaN'
    return val

#please specify a multiple of 3h for the time
def get_max_wind_change(data, time):
    df = data
    df['max_wind_change']=[get_max_change(data, time, i) for i in range(len(data))]
    return df

In [7]:
#to use in the future: computes the wind category
def sust_wind_to_cat_one_hot(wind):
    # maximum sustained wind in kt (knot)
    if wind<=33: cat='TD' # <=33
    elif wind<=63.:  cat='TS'
    elif wind <=82.: cat='H1'
    elif wind <=95.: cat='H2'
    elif wind <=112.: cat='H3'
    elif wind <=136.: cat='H4'    
    elif wind > 136. : cat='H5'
    else: cat = 'nan'  

    return cat

def sust_wind_to_cat_val(wind):
    # maximum sustained wind in kt (knot)
    if wind<=33: cat= 0 # <=33
    elif wind<=63.:  cat=1
    elif wind <=82.: cat=2
    elif wind <=95.: cat=3
    elif wind <=112.: cat=4
    elif wind <=136.: cat=5    
    elif wind > 136. : cat=6
    else: cat = 0  

    return cat

In [8]:
def add_storm_category_one_hot(data):
    df = pd.DataFrame()
    df['storm_category'] = [sust_wind_to_cat_one_hot(data['WMO_WIND'][i]) for i in range(len(data))]
    storm_cat = pd.get_dummies(df['storm_category'],prefix='storm_category')
    #storm_cat
    storm_cat.drop('storm_category_nan', axis=1, inplace=True)
    frames = [data, storm_cat]
    df0 = pd.concat(frames, axis = 1)
    #df0.drop('storm_category', axis=1)
    print("Storm category is now added and one-hot.")
    return df0

def add_storm_category_val(data):
    df = pd.DataFrame()
    df['storm_category'] = [sust_wind_to_cat_val(data['WMO_WIND'][i]) for i in range(len(data))]
    frames = [data, df]
    df0 = pd.concat(frames, axis = 1)
    #df0.drop('storm_category', axis=1)
    return df0

In [9]:
def sort_storm(data, min_wind, min_steps = 5, max_steps = 120):
    '''function to create dictionary of storm matrices
    arguments: 
    data we want to cut
    min_wind: the minimum wind speed to store data 
    '''
    #get unique storm_id:
    SID=pd.unique(data['SID']).tolist()
    #remove empty SID
    #if not dropna: SID.remove(' ') 
    #create empty dictionary
    dict0={}
    ind = 0
    for i in range(len(SID)):
        #get data of a particular SID
        M = data.loc[data['SID'] == SID[i]]
        #cut off using min wind speed
        #TODO : cut everything before, ie look for the right date
        try:
            t = M.index[M['WMO_WIND']>= min_wind][0]
            t0 = M.index[0]
        except:
            t = 0
        N = M.loc[M['WMO_WIND'] >= min_wind]
        #save matrix in dict0
        if N.shape[0] > min_steps:
            ind+=1
            dict0.update({ind:M.iloc[t-t0:max_steps+t-t0]})
    print("The dictionary of storms has been created.")
    return dict0

#storms is the dictionary all storm matrices
#storms = sort_storm(df0, min_wind=45, min_steps=15)

In [10]:
#Geographical difference features: i.e. feature_1(t) = feature(t)-feature(0)
    # features: LAT, LON, DIST2LAND
def geo_diff(dict0):
    dict1={}
    #loop over each dataframe
    for i in dict0:
        df=dict0[i]
        #reset index
        df.reset_index(inplace=True, drop=True)
        #calculate difference from t=0 
        df['LAT_1']= df['LAT'] - df['LAT'][0]
        df['LON_1']= df['LON'] - df['LON'][0]
        df['DIST2LAND_1']= df['DIST2LAND'] - df['DIST2LAND'][0]
        #substitute back to the dictionary
        dict1[i]=df
    return dict1

In [11]:
#instead of padding with 0, pad with latest values in loop
def pad_traj(dict0, max_steps, nan = False):
    dict1={}
    for t in dict0:
        num_steps = dict0[t].shape[0]
        steps2add = max_steps - num_steps
        if steps2add > 0:
            if nan:
                dict1[t] = pd.concat([dict0[t], pd.DataFrame([[np.nan] * dict0[t].shape[1]]*steps2add, columns=dict0[t].columns)], ignore_index=True)
            else:
                dict1[t] = pd.concat([dict0[t], pd.DataFrame([[0] * dict0[t].shape[1]]*steps2add, columns=dict0[t].columns)], ignore_index=True)                
                #In fact it happens to be easier to make the change afterwards with repad
                #dict1[t] = pd.concat([dict0[t], pd.DataFrame([dict0[t].tail(1)]*steps2add, columns=dict0[t].columns)], ignore_index=True)                       
        else:
            dict1[t] = dict0[t][:max_steps]
    print("The trajectories have now been padded.")
    return dict1

In [12]:
def get_distance_km(lon1, lat1, lon2, lat2):
    '''
    Using haversine formula (https://www.movable-type.co.uk/scripts/latlong.html)
    '''
    R=6371e3 # meters (earth's radius)
    phi_1=math.radians(lat1)
    phi_2 = math.radians(lat2)
    delta_phi=math.radians(lat2-lat1)
    delta_lambda=math.radians(lon2-lon1)
    a=np.power(math.sin(delta_phi/2),2) + math.cos(phi_1)*math.cos(phi_2)\
      * np.power(math.sin(delta_lambda/2),2)
    c= 2 * math.atan2(math.sqrt(a),math.sqrt(1-a))

    return R*c/1000.

#compute the displacement from t=0
def add_displacement_distance(dict0):
    dict1={}
    #loop over each dataframe
    for i in dict0:
        df=dict0[i]
        #reset index
        df.reset_index(inplace=True, drop=True)
        #calculate difference from t=0 
        df['DISPLACEMENT'] = 0
        for j in range(1,len(df)):
            d = get_distance_km(df['LON'][j-1], df['LAT'][j-1], df['LON'][j], df['LAT'][j])
            if d > 500: d=0
            df['DISPLACEMENT'][j] = d 
        dict1[i]=df
    return dict1

def add_displacement_lat_lon2(dict0):
    dict1={}
    #loop over each dataframe
    for i in dict0:
        df=dict0[i]
        #reset index
        df.reset_index(inplace=True, drop=True)
        lst_lat = [0]
        lst_lon = [0]
        for j in range(1,len(df)):
            d_lat = df['LAT'][j] - df['LAT'][j-1]
            d_lon = df['LON'][j] - df['LON'][j-1]
            lst_lat.append(d_lat)
            lst_lon.append(d_lon)
        df['DISPLACEMENT_LAT'] = lst_lat
        df['DISPLACEMENT_LON'] = lst_lon
        dict1[i]=df
    return dict1

In [13]:
#function to calculate tensor shape
    #input: dictionary of storm data
def tensor_shape(dict0):
    #number of storms
    num_storms=len(dict0) - 1
    #number of features
    num_features=dict0[next(iter(dict0))].shape[1]  
    
    #to compute min and max number of steps
    t_max = 0 #initialise 
    t_min = 1000
    t_hist = []
    for i in dict0:
        t0 = dict0[i].shape[0]
        t_hist.append(t0)
        if  t0 > t_max:
            t_max = t0
        if t0 < t_min:
            t_min = t0
    print("There are %s storms with %s features, and maximum number of steps is %s and minimum is %s." %(num_storms,num_features,t_max, t_min))
    return num_storms, num_features, t_max, t_min, t_hist     
    
#call tensor_shape 
#m, n, t_max, t_min, t_hist = tensor_shape(d)

In [14]:
#create a tensor
def create_tensor(data, number_of_storms):
    tensor = data[1]
    for i in range(2,number_of_storms,1):
        tensor=np.dstack((tensor, data[i]))
    #return list of features 
    p_list = data[1].columns.tolist()
    return tensor, p_list

def repad(t):
    for i in range(t.shape[0]):
        if t[i][2][-1] == 0:
            ind = np.argmin(t[i][2])
            for j in range(ind,t.shape[2]):
                t[i,:,j]=t[i,:,ind-1]
    return t

In [41]:
def prepare_data(path = "./data/ibtracs.last3years.list.v04r00.csv", max_wind_change = 12, min_wind = 50, min_steps = 15, max_steps = 120, secondary = False, one_hot=False, dropna = False):
    data = pd.read_csv(path) 
    #select interesting columns
    df0 = select_data(data)
    #transform data from String to numeric
    df0 = numeric_data(df0)
    #add cos & sign of the day
    df0 = smooth_day(df0)
    #if dropna: df0 = df0.dropna()
    #add one_hot columns:
    if one_hot: 
        #add one-hot storm category
        #df0 = add_storm_category_val(df0)   
        df0 = add_storm_category_one_hot(df0)
        #transform basin and nature of the storm into one-hot vector
        df0 = add_one_hot(data, df0)
    if secondary: 
        #add the max-wind-change column
        df0 = get_max_wind_change(df0, max_wind_change)
    
    #get a dict with the storms with a windspeed greater to a threshold
    storms = sort_storm(df0, min_wind, min_steps)
    #pad the trajectories to a fix length
    d = pad_traj(storms, max_steps)
    #print(d)
    if secondary:
        #d = add_displacement_distance(d)
        d = add_displacement_lat_lon2(d)
    #print the shape of the tensor
    m, n, t_max, t_min, t_hist = tensor_shape(d)
    #create the tensor
    t, p_list = create_tensor(d, m)
    #delete id and number of the storms
    t2 = torch.Tensor(t[:,3:,:].astype('float64'))
    #match feature list 
    p_list = p_list[3:]
    #transpose time and sample
    t3=torch.transpose(t2,0,2)
    #replace 0 by latest values in the tensor
    t3 = repad(t3)
    return t3, p_list

In [19]:
def prepare_data2(path = "./data/ibtracs.last3years.list.v04r00.csv", max_wind_change = 12, min_wind = 50, min_steps = 15, max_steps = 120, secondary = False, one_hot=False, dropna = False):
    data = pd.read_csv(path) 
    #select interesting columns
    df0 = select_data(data)
    #transform data from String to numeric
    df0 = numeric_data(df0)
    #if dropna: df0 = df0.dropna()
    #add one_hot columns:
    if one_hot: 
        #add one-hot storm category
        #df0 = add_storm_category_val(df0)   
        df0 = add_storm_category_one_hot(df0)
        #transform basin and nature of the storm into one-hot vector
        df0 = add_one_hot(data, df0)
    if secondary: 
        #add the max-wind-change column
        df0 = get_max_wind_change(df0, max_wind_change)
    
    #get a dict with the storms with a windspeed greater to a threshold
    storms = sort_storm(df0, min_wind, min_steps)
    #pad the trajectories to a fix length
    d = pad_traj(storms, max_steps)
    #print(d)
    if secondary:
        #d = add_displacement_distance(d)
        d = add_displacement_lat_lon2(d)
    #print the shape of the tensor
    m, n, t_max, t_min, t_hist = tensor_shape(d)
    #create the tensor
    t, p_list = create_tensor(d, m)
    #delete id and number of the storms
    #t2 = torch.Tensor(t[:,1:3,:])#.astype('float64'))
    #match feature list 
    #p_list = p_list[3:]
    #transpose time and sample
    #t3=torch.transpose(t2,0,2)
    #replace 0 by latest values in the tensor
    #t3 = repad(t3)
    return t[:,2:5,:], p_list

In [20]:
t3, p_list = prepare_data2(min_wind = 60, min_steps = 60, max_steps=60, one_hot = False, secondary = False, dropna = True)

The dictionary of storms has been created.
The trajectories have now been padded.
There are 11 storms with 9 features, and maximum number of steps is 60 and minimum is 60.


In [ ]:
t3, p_list = prepare_data(path = "ibtracs.since1980.list.v04r00.csv", min_wind = 60, min_steps = 1, max_steps=60, one_hot = False, secondary = True, dropna = True)
#d = prepare_data2(min_wind = 120, min_steps = 1, max_steps=60, one_hot = False, secondary = True, dropna = True)

The dictionary of storms has been created.
The trajectories have now been padded.
There are 1830 storms with 12 features, and maximum number of steps is 60 and minimum is 60.


In [ ]:
#2 dimensional SVD 
def mat_svd(X,r,m,n,t):
    #Mode-1 folding
    X1= X.reshape(m,n*t)
    U,S,V= torch.svd(X1,some=True, compute_uv=True) 
    U1=U[:,:r]
    #Mode-2 folding
    X2= X.reshape(m*t,n) 
    U,S,V= torch.svd(X2,some=True, compute_uv=True) 
    V1=V[:,:r]
    return U1,V1

#calculate MAE:
def get_mae(tensor1,tensor2):
    error = (tensor1-tensor2).abs().sum().item()
    return error 


#Slice Iteration: for one iteration 
def slice_iteration(tensor,r,m,n,t):
    new_tensor = torch.empty((m,n,t)) 
    #get U,V
    U,V = mat_svd(tensor, r,m,n,t)
    
    #update tensor 
    UU = torch.mm(U,torch.t(U))
    VV = torch.mm(V,torch.t(V))
    for i in range(t):
        new_tensor[:,:,i] = torch.mm(torch.mm(UU, tensor[:,:,i]),VV) 
    return new_tensor 

In [ ]:
#Slice Learning 
def slice_learning(tensor_train,r,max_iteration=50,convergence=0.001, plot=False):
    m,n,t = tensor_train.shape
    new_tensor = torch.empty(m,n,t) 
    old_tensor = tensor_train 
    mae_all=[]
    
    mae = convergence + 1 
    #Iterative slice learning 
    for i in range(max_iteration): 
        if mae > convergence: 
            new_tensor = slice_iteration(old_tensor,r,m,n,t)
            mae = get_mae(new_tensor,old_tensor) 
            mae_all.append(mae)
            old_tensor = new_tensor 
    
    #plot MAE difference between iterations   
    max_i = len(mae_all)

    print('Code stopped at iteration = %s, and mae difference = %s'%(max_i,mae_all[-1]))
    if plot:
        plt.plot(np.linspace(1,max_i,max_i),mae_all)
        plt.yscale('log')
        plt.show()
    return new_tensor 

#X_imputed = slice_learning(t3_test2,16)

#find the MAE difference between X_imputed and X_original
#print('The MAE between X_imputed and X_original is %s' %(get_mae(X_imputed,t3_test2)))

In [ ]:
#Function to standardize and de_standardize data 
def standardize(tensor): 
    new_tensor = tensor.clone()
    #calculate along 2nd (python index) dimension 
    #std, mean = torch.std_mean(tensor[:,:k,:], dim=2)
    # @cynthia: I replaced your line by this one so that it works with my torch version
    std, mean = torch.std(tensor[:,:,:], dim=2), torch.mean(tensor[:,:,:], dim=2)
    t=tensor.shape[2]
    for i in range(t):
        #element wise division
        new_tensor[:,:,i] = torch.div((tensor[:,:,i] - mean),std)  
    #replace Nan with 0 
    new_tensor[new_tensor != new_tensor] = 0
    return new_tensor, mean, std

def de_standardize(tensor,mean,std):
    t=tensor.shape[2]
    new_tensor = tensor.clone()
    for i in range(t):
        #element wise multiplication
        new_tensor[:,:,i] = torch.mul(tensor[:,:,i] ,std) + mean 
    return new_tensor 

# a,m,std=standardize(x)
# b=de_standardize(a,m,std) 
# print('standardize',a )
# print('de_standardize', b)
   

In [ ]:
#Methods to standardize and de_standardize data along samples and timesteps (full standardization)
def standardize2(tensor): 
    new_tensor = tensor.clone()
    #calculate along 0 and 2nd (python index) dimension 
    std, mean = torch.std(tensor, dim=[0,2]), torch.mean(tensor, dim=[0,2])
    t = tensor.shape[1]
    for i in range(t):
        #element wise division
        new_tensor[:,i,:] = torch.div((tensor[:,i,:] - mean[i]), std[i])  
    #replace Nan with 0 
    new_tensor[new_tensor != new_tensor] = 0
    return new_tensor, mean, std

def de_standardize2(tensor, mean, std):
    t = tensor.shape[1]
    new_tensor = tensor.clone()
    for i in range(t):
        #element wise multiplication
        new_tensor[:,i,:] = torch.mul(tensor[:,i,:], std[i]) + mean[i] 
    return new_tensor 

In [ ]:
#plot error bar chart
def plot_error(data, X_imputed, storm_id, p_list):#calculate error for all features for next 5 hour prediction: 
    error_5=[]
    error_10=[]
    error_15=[]
    for p in range(len(p_list)):
        #for each time
        error_v=np.abs(X_imputed[storm_id][p] - data[storm_id][p])
        error_5.append((error_v[time+5-1:time+5+1].sum()*(1/3)))#avg of time +5
        error_10.append((error_v[time+10-1:time+10+1].sum()*(1/3)))#avg of time +10
        error_15.append((error_v[time+15-1:time+15+1].sum()*(1/3)))#avg of time +15
    
    #plotting error:
    ind = np.arange(len(p_list)) 
    width = 0.3 
    plt.bar(ind,error_5,width, label='t+5')
    plt.bar(ind+width,error_10,width, label='t+10')
    plt.bar(ind+2*width,error_15,width, label='t+15')
    plt.xticks(ind + width / 2, p_list)
    plt.title('Avg error for t+5/10/15 StormID=%s' %( storm_id))
    plt.setp(plt.gca().get_xticklabels(), rotation=45, horizontalalignment='right')
    plt.legend(loc='best')
    plt.show()
    return 

def plot_feature_prediction(data, X_imputed_0, X_imputed_1, X_imputed_2, storm_id, p_list, p_range):
    for p in p_range:
        #plot predicted feature p with various normalization methods: 
        l = [i for i in range(data.shape[2])]
        plt.plot(l, np.array(X_imputed_0[storm_id][p]), label='imputed_no_scale')
        plt.plot(l, np.array(X_imputed_1[storm_id][p]),label='imputed_normalize')
        plt.plot(l, np.array(X_imputed_2[storm_id][p]), label='imputed_standardize')
        plt.plot(l, np.array(data[storm_id][p]),label='real')
        plt.axvline(x=time)
        plt.legend()
        plt.title('Feature= %s, StormID=%s' %(p_list[p],storm_id))
        plt.show()
    return 

In [ ]:
#Testing function by deleting entries after time t for one particular storm  
def test(data, p_list, storm_id, time, r, p = 2, delete = True):
    data0 = data.clone()
    
    #data0= delete one storm data for testing  
    if delete:    
        for t in range(time,data.shape[2]):
            data0[storm_id,:,t] = data0[storm_id,:,time-1]
    
    ### slice_learning 0: no_scale 
    X_imputed_0 = slice_learning(data0,r)
    
    ### slice learning 1: normalize data 
    data1 = F.normalize(data0, dim = 2)
    X_imputed_1 = slice_learning(data1,r)
    #de_normalize
    scale = torch.div(data[:,:,:time],X_imputed_1[:,:,:time])
    scale_mean = torch.mean(scale, dim=2)
    for i in range(X_imputed_1.shape[2]):
        X_imputed_1[:,:,i] = torch.mul(X_imputed_1[:,:,i], scale_mean) 
    
    ### slice_learning 2: standardize
    data2, mean, std = standardize2(data0)
    X_imputed_2 = slice_learning(data2, r)
    #de_standardize 
    X_imputed_2= de_standardize2(X_imputed_2, mean, std)
    
    return X_imputed_0, X_imputed_1, X_imputed_2

In [23]:
t3, p_list = prepare_data(path = "ibtracs.since1980.list.v04r00.csv", min_wind = 50, min_steps= 59, max_steps=60, one_hot = False, secondary = False)

The dictionary of storms has been created.
The trajectories have now been padded.
There are 344 storms with 9 features, and maximum number of steps is 60 and minimum is 60.


In [ ]:
#60 is big storm not easy to predict
storm_id = 340
time = 20
r = 13 #size of latent features 
for r in [11]:
    print ('r =', r)
    #test learning for a particular storm_id 
    X_imputed_0, X_imputed_1, X_imputed_2 = test(t3, p_list, storm_id, time, r, delete = False)
    #make some plots:
    plot_error(t3, X_imputed_2, storm_id, p_list)
    plot_feature_prediction(t3, X_imputed_0, X_imputed_1, X_imputed_2, storm_id, p_list, p_range=[2])

In [ ]:
#get best r
def get_best_r(data, r_min, r_max):
    r_list = []
    for r in range(r_min, r_max):
        X_imputed = slice_learning(data,r, plot = False)
        r_list.append(get_mae(X_imputed,data))
    plt.plot([r for r in range(r_min, r_max)], r_list)
    plt.show()
    return r_list

get_best_r(t3[:,2:22,:], 5, 21)

In [ ]:
r=5
m,n,t=t3.shape

#SVD 
def mat_svd(X,r):
    #Mode-1 folding
    X1= X.reshape(m,n*t)
    U,S,V= torch.svd(X1,some=True, compute_uv=True) 
    U1=U[:,:r]
    #Mode-2 folding
    X2= X.reshape(m*t,n) 
    U,S,V= torch.svd(X2,some=True, compute_uv=True)
    V1=V[:,:r]
    return U1,V1

#Update S based on U1, V1
def update_s(U, X, V, r ):
    new_s = torch.empty((r,r,t))
    for i in range(t): 
        m1 = torch.mm(torch.t(U),X[:,:,i]) #torch.t = transpose
        m2 = torch.mm(m1,V)
        new_s[:,:,i]=m2
    return new_s

#Compute Z:
def update_z(U,V,S):
    new_z = torch.empty((m,n,t))
    for i in range(t): 
        S_t=S[:,:,i]
        m1 = torch.mm(U,S_t)
        m2 = torch.mm(m1,torch.t(V))
        new_z[:,:,i]=m2
    return new_z

#wrap up everything 
def tensor_svd1(Z0,r,T=50):
    
    #initialise R
    R=torch.rand(Z0.shape)
    error=100
    error_all=[]
    
    for t in range(T):
        if error > 1: 
            U1,V1= mat_svd(R,r)
            S=update_s(U1,R,V1,r)
            Z1=update_z(U1,V1,S)
            R=Z1
            #calculate error 
            error=(Z1-Z0).pow(2).sum().item()
            error_all.append(error)
    
    print('code stopped at t=%s at error=%s'%(t,error))
    
    #plot error 
    L=len(error_all)
    plt.plot(np.linspace(1,L,L),error_all)
    plt.show()
    
    return Z1    


In [ ]:
tensor_svd1(t3,5)

In [ ]:
x = torch.rand(2,3,4)
print('x=',x)
#     m,n,t = x.shape
#     mean = torch.zeros(m,n)
#     std = torch.zeros(m,n)
#     for i in range(m):
#     for j in range(n):
#         mean[i,j] = x[i,j,:].mean()
#         std[i,j] = x[i,j,:].std() 
#     print ('loop result',std,mean)

#calculate standard deviation and mean of tensor
def standardize(tensor):
    new_tensor = tensor.clone()
    #calculate along 2nd (python index) dimension 
    std, mean = torch.std_mean(tensor, dim=2)
    t=tensor.shape[2]
    for i in range(t):
        #element wise division
        new_tensor[:,:,i] = torch.div((tensor[:,:,i] - mean),std) 
    return new_tensor, mean, std

def de_standardize(tensor,mean,std):
    t=tensor.shape[2]
    new_tensor = tensor.clone()
    for i in range(t):
        #element wise multiplication
        new_tensor[:,:,i] = torch.mul(tensor[:,:,i] ,std) + mean 
    return new_tensor 
a,m,std=standardize(x)
b=de_standardize(a,m,std) 
print('standardize',a )
print('de_standardize', b)
 
    
torch.svd(a,some=True, compute_uv=True)

In [ ]:
#to use in the future: computes the wind category
def sust_wind_to_cat(wind):
    # maximum sustained wind in kt (knot)
    if wind<=33: cat='TD' # <=33
    elif wind<=63.:  cat='TS'
    elif wind <=82.: cat='H1'
    elif wind <=95.: cat='H2'
    elif wind <=112.: cat='H3'
    elif wind <=136.: cat='H4'
    else: cat='H5'

    return cat

In [ ]:
#allows to keep only the data between two dates to specify
def get_periodic_data(data, start ='2018-01-01',end ='2018-12-31',mincat=None,maxcat=None):
    n = len(data)
    start = datetime.strptime(start, '%Y-%m-%d')
    end = datetime.strptime(end, '%Y-%m-%d')
    times = pd.to_datetime(data['ISO_TIME'][1:n])
    ind_start = 1
    ind_end = n
    i = 1
    while times[i] < start :#or i < n:
        i+=1
    ind_start = i
    while times[i] < end :#or i < n:
        i+=1
    ind_end = i
    return data[ind_start:ind_end]

In [ ]:
#allows to get the distinct ids of the storms 
def get_ids(data):
    return data.SID.unique()

In [ ]:
def get_storm_track(data, storm_id = '2016005N02187', time_step = 6):
    #data.set_index("SID", inplace=True)
    return data.loc[data['SID'] == storm_id]

In [ ]:
storm = get_storm_track(data)
lat = np.array(storm['LAT'])
lat2 = [float(lat[i]) for i in range(len(lat))]
lon = np.array(storm['LON'])
lon2 = [float(lon[i]) for i in range(len(lon))]

In [ ]:
plt.plot(lat2, lon2)
plt.show()

Some functions we may use

In [ ]:
def get_distance_km(lon1, lat1, lon2, lat2):
    '''
    Using haversine formula (https://www.movable-type.co.uk/scripts/latlong.html)
    '''
    R=6371e3 # meters (earth's radius)
    phi_1=math.radians(lat1)
    phi_2 = math.radians(lat2)
    delta_phi=math.radians(lat2-lat1)
    delta_lambda=math.radians(lon2-lon1)
    a=np.power(math.sin(delta_phi/2),2) + math.cos(phi_1)*math.cos(phi_2)\
      * np.power(math.sin(delta_lambda/2),2)
    c= 2 * math.atan2(math.sqrt(a),math.sqrt(1-a))

    return R*c/1000.


def get_longlat_from_offsets(lon, lat, dkm_lon, dkm_lat):
    '''
    :param lon: initial longitude
    :param lat: inital latitude
    :param dn:     offsets in meters
    :param de:     offsets in meters
    :return: lon_final, lat_final
    '''
    # Earth’s radius, sphere
    R = 6378137
    # Coordinate offsets in radians
    dLat = dkm_lat / R
    dLon = dkm_lon / (R * math.cos(math.radians(lat)))

    # OffsetPosition, decimal degrees
    latO = lat + dLat * 180 / math.pi
    lonO = lon + dLon * 180 / math.pi

    return lonO, latO